In [22]:
from dotenv import load_dotenv
import os

load_dotenv()

## GPT 3.5 Judge

In [20]:
from openai import OpenAI

openai_api_key= os.getenv('OPENAI_API_KEY')
if openai_api_key is None:
    raise ValueError("API Key is not set.")

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
    )
    return chat_completion

def is_code_openai(row):
    text = row.str.cat(sep='\n')
    prompt = f"""
            I need you to analyze a question answer sequence of text and determine whether it contains code written 
            in any programming language. Please respond with "Yes" if the text contains code and 
            "No" if it does not. All langauages are valid, including markup languages.
            Example 1:
                Input: 
                    def hello_world():
                        print("Hello, world!")
                Output:
                    Yes
            Example 2:
                Input:
                    The quick brown fox jumps over the lazy dog.
                Output:
                    No

            It is important that you only answer with "Yes" or "No". No further explanation or justification
            is needed. You should be robust enough to handle code snippets in any programming language.
            Now, analyze the following text:
                Input:
                    {text}
                Output:         
            """
    answer = get_completion(prompt).choices[0].message.content
    return answer

## LLama 3

In [24]:
import transformers
import torch

huggingface_access_token = os.getenv('HUGGINGFACE_ACCESS_TOKEN')
if huggingface_access_token is None:
    raise ValueError("Huggingface access token is not set.")

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto",
    use_auth_token=huggingface_access_token
)
pipeline("Hey how are you doing today?")

/Users/chadbrouze/Library/Caches/pypoetry/virtualenvs/scripts-1_evkgVF-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
401 Client Error. (Request ID: Root=1-6672965e-0060b90d63388f32400d314f;74145b34-1e34-4663-bc21-53ad848a0e3b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it.

## Main

In [19]:
from datasets import load_dataset
import pandas as pd
import os
import time

dataset_name = "yahma/alpaca-cleaned"
dataset = load_dataset(dataset_name)
df = pd.DataFrame(dataset['train'])
new_order = ['instruction', 'input', 'output']
df = df[new_order]
column_names = df.columns
     
for i, row in df.iterrows():
    if i < 6129:
        continue
    print(is_code_openai(row))
    time.sleep(0.2)

    

No
No
No
No
No
Yes
No
No
No


KeyboardInterrupt: 